### Checking Versions

First, let's check which versions of Python and Spark we're using:

In [1]:
# Print Python and Spark versions
import sys
from pyspark.sql import SparkSession

# Create a SparkSession first
spark = SparkSession.builder.appName("Showdown-Replay-Exploration").getOrCreate()

print(f"Python version: {sys.version}")
print(f"Spark version: {spark.version}")

Python version: 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
Spark version: 3.5.0


### Step 1: Setup and Load JSON Files

We will load only the Regulation G Json files.

In [2]:
# Use spark to read files then partition after to speed up processing

# Read JSON files from the directory recursively
logs_df = spark.read \
    .option("recursiveFileLookup", "true") \
    .option("pathGlobFilter", "*.json") \
    .json("../../data/replays/gen9vgc2025regg/raw/2024-12-02/") \
    .limit(1000)  # Only take the first 1000 records


# Optionally repartition to merge many small files; adjust number based on your cluster and data size.
logs_df = logs_df.repartition(8)

### Step 2: Extract Relevant Information

Player Information

In [3]:
from pyspark.sql.functions import regexp_extract, transform, expr, regexp_replace, col, explode, split, lit
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

# Extract player names and ratings
logs_df = logs_df.withColumn("player1", logs_df["players"].getItem(0))
logs_df = logs_df.withColumn("player2", logs_df["players"].getItem(1))

# For player1_rating_before and player2_rating_before
logs_df = logs_df.withColumn("player1_rating_before", regexp_extract("log", r"\{\}'s rating: (\d+)", 1))
logs_df = logs_df.withColumn("player2_rating_before", regexp_extract("log", r"\{\}'s rating: (\d+)", 1))

# For player1_rating_after and player2_rating_after
logs_df = logs_df.withColumn("player1_rating_after", regexp_extract("log", r"\{\}'s rating: \d+ → (\d+)", 1))
logs_df = logs_df.withColumn("player2_rating_after", regexp_extract("log", r"\{\}'s rating: \d+ → (\d+)", 1))

Team Sheet data

If both players accept open team sheets, we can get a bunch of information without having to parse all the logs. We will check if open team sheets were accepted, and extract that data from those games

In [4]:


def analyze_team_sheets(df):
    from pyspark.sql.functions import regexp_extract, transform, expr, regexp_replace, col, explode, split, lit
    from pyspark.sql.functions import collect_list, struct, map_from_entries, when
    
    # Check if team sheets were accepted (presence of showteam indicates acceptance)
    df_with_acceptance = df.withColumn(
        "team_sheets_accepted", 
        when(col("log").contains("|showteam|p"), lit(True)).otherwise(lit(False))
    )
    
    # Extract showteam information when accepted
    df_with_team_info = df_with_acceptance.withColumn(
        "p1_team_details", 
        when(col("log").contains("|showteam|p"), 
             regexp_extract(col("log"), r"\|showteam\|p1\|(.*?)(?=\n|\|showteam\|p2\|)", 1))
        .otherwise(lit(None))
    ).withColumn(
        "p2_team_details", 
        when(col("log").contains("|showteam|p2|"), 
             regexp_extract(col("log"), r"\|showteam\|p2\|(.*?)(?=\n|\|j\||\|inactive\|)", 1))
        .otherwise(lit(None))
    )
    
    # Process player 1 team details
    df_p1 = df_with_team_info.filter(col("p1_team_details").isNotNull())
    if df_p1.count() > 0:
        # Split the team details into individual Pokémon entries
        df_p1 = df_p1.withColumn("p1_team_entries", split(col("p1_team_details"), r"\]"))
        
        # Explode the entries to process each Pokémon
        df_p1_exploded = df_p1.select("id", explode("p1_team_entries").alias("p1_entry"))
        
        # Extract details for each Pokémon
        df_p1_pokemon = df_p1_exploded.withColumn(
            "pokemon", regexp_extract(col("p1_entry"), r"^(.*?)\|\|", 1)
        ).withColumn(
            "item", regexp_extract(col("p1_entry"), r"\|\|(.*?)\|", 1)
        ).withColumn(
            "ability", regexp_extract(col("p1_entry"), r"\|([^|]+)\|[^|]+,[^|]+,", 1)
        ).withColumn(
            "moves", regexp_extract(col("p1_entry"), r"\|([^|]+,[^|]+,[^|]+,[^|]+)\|", 1)
        ).withColumn(
            "tera_type", regexp_extract(col("p1_entry"), r",,,,,([^,\]]+)", 1)
        )
        
        # Filter out empty entries
        df_p1_pokemon = df_p1_pokemon.filter(col("pokemon").isNotNull() & (col("pokemon") != ""))
        
        # Create a map of Pokémon to their details
        p1_pokemon_map = df_p1_pokemon.groupBy("id").agg(
            map_from_entries(
                collect_list(
                    struct(
                        col("pokemon"), 
                        struct(
                            col("item"), 
                            col("ability"), 
                            col("moves"), 
                            col("tera_type")
                        )
                    )
                )
            ).alias("p1_pokemon_data_open_team")
        )
        
        # Join back to the main dataframe
        df_with_team_info = df_with_team_info.join(p1_pokemon_map, "id", "left")
    else:
        # Add empty map if no data
        df_with_team_info = df_with_team_info.withColumn("p1_pokemon_data", lit(None))
    
    # Process player 2 team details
    df_p2 = df_with_team_info.filter(col("p2_team_details").isNotNull())
    if df_p2.count() > 0:
        # Split the team details into individual Pokémon entries
        df_p2 = df_p2.withColumn("p2_team_entries", split(col("p2_team_details"), r"\]"))
        
        # Explode the entries to process each Pokémon
        df_p2_exploded = df_p2.select("id", explode("p2_team_entries").alias("p2_entry"))
        
        # Extract details for each Pokémon
        df_p2_pokemon = df_p2_exploded.withColumn(
            "pokemon", regexp_extract(col("p2_entry"), r"^(.*?)\|\|", 1)
        ).withColumn(
            "item", regexp_extract(col("p2_entry"), r"\|\|(.*?)\|", 1)
        ).withColumn(
            "ability", regexp_extract(col("p2_entry"), r"\|([^|]+)\|[^|]+,[^|]+,", 1)
        ).withColumn(
            "moves", regexp_extract(col("p2_entry"), r"\|([^|]+,[^|]+,[^|]+,[^|]+)\|", 1)
        ).withColumn(
            "tera_type", regexp_extract(col("p2_entry"), r",,,,,([^,\]]+)", 1)
        )
        
        # Filter out empty entries
        df_p2_pokemon = df_p2_pokemon.filter(col("pokemon").isNotNull() & (col("pokemon") != ""))
        
        # Create a map of Pokémon to their details
        p2_pokemon_map = df_p2_pokemon.groupBy("id").agg(
            map_from_entries(
                collect_list(
                    struct(
                        col("pokemon"), 
                        struct(
                            col("item"), 
                            col("ability"), 
                            col("moves"), 
                            col("tera_type")
                        )
                    )
                )
            ).alias("p2_pokemon_data_open_team")
        )
        
        # Join back to the main dataframe
        df_with_team_info = df_with_team_info.join(p2_pokemon_map, "id", "left")
    else:
        # Add empty map if no data
        df_with_team_info = df_with_team_info.withColumn("p2_pokemon_data", lit(None))
        
    # Drop the temporary columns before returning
    final_df = df_with_team_info.drop("p1_team_details", "p2_team_details")
    
    
    return final_df

# Call the function
logs_df_with_teamsheet = analyze_team_sheets(logs_df)

# Call the function
# logs_df2 = analyze_team_sheets(logs_df)

pandas_df_preview1 = logs_df_with_teamsheet.limit(15).toPandas()

pandas_df_preview1


,id,format,formatid,log,password,players,private,rating,uploadtime,views,player1,player2,player1_rating_before,player2_rating_before,player1_rating_after,player2_rating_after,team_sheets_accepted,p1_pokemon_data_open_team,p2_pokemon_data_open_team
0,gen9vgc2025regg-2255482030,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆video gamah\n|j|☆goom_n_gloomba\n|t:|17331...,None,"[video gamah, goom_n_gloomba]",0,1064.0,1733179523,20,video gamah,goom_n_gloomba,,,,,False,None,None
1,gen9vgc2025regg-2255265255,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆greenbean98\n|j|☆starkhd\n|t:|1733158007\n...,None,"[greenbean98, starkhd]",0,1049.0,1733158270,6,greenbean98,starkhd,,,,,False,None,None
2,gen9vgc2025regg-2254987654,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆tsilva22\n|j|☆aroswing\n|t:|1733117542\n|g...,None,"[tsilva22, aroswing]",0,1095.0,1733117753,3,tsilva22,aroswing,,,,,False,None,None
3,gen9vgc2025regg-2255274377,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Freaky Mane\n|j|☆Bhdidi\n|html|<table widt...,None,"[Freaky Mane, Bhdidi]",0,NaN,1733159149,3,Freaky Mane,Bhdidi,,,,,True,"{'Flutter Mane': ('AssaultVest', 'Protosynthes...","{'Flutter Mane': ('ChoiceSpecs', 'Protosynthes..."
4,gen9vgc2025regg-2255415209,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆GriefersVGC\n|j|☆johnnydrama219\n|t:|17331...,None,"[GriefersVGC, johnnydrama219]",0,1049.0,1733172341,9,GriefersVGC,johnnydrama219,,,,,False,None,None
5,gen9vgc2025regg-2254867190,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Mumu_the_Swampert\n|j|☆alejaji\n|t:|173310...,None,"[Mumu_the_Swampert, alejaji]",0,1085.0,1733100987,13,Mumu_the_Swampert,alejaji,,,,,False,None,None
6,gen9vgc2025regg-2255426521,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆groovalishis\n|j|☆juacox300\n|t:|173317328...,None,"[groovalishis, juacox300]",0,1089.0,1733173539,51,groovalishis,juacox300,,,,,False,None,None
7,gen9vgc2025regg-2255448421,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Mr.Man1206\n|j|☆lukve_\n|t:|1733175480\n|g...,None,"[Mr.Man1206, lukve_]",0,1075.0,1733175763,10,Mr.Man1206,lukve_,,,,,False,None,None
8,gen9vgc2025regg-2255280504,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆heavensfirstsin\n|j|☆locomobius\n|t:|17331...,None,"[heavensfirstsin, locomobius]",0,1000.0,1733159813,11,heavensfirstsin,locomobius,,,,,False,None,None
9,gen9vgc2025regg-2255105966,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆peepeeyro\n|j|☆Trixiquis\n|t:|1733139466\n...,None,"[peepeeyro, Trixiquis]",0,1000.0,1733139704,3,peepeeyro,Trixiquis,,,,,False,None,None


Pokémon, Moves, and Items

In [5]:
# Extract all Pokémon for each player
def extract_raw_data(df):
    df = df.withColumn("player1_pokemon_array", 
                                 expr("regexp_extract_all(log, '\\\\|poke\\\\|p1\\\\|([^\\\\|]+)', 1)"))
    df = df.withColumn("player2_pokemon_array", 
                                 expr("regexp_extract_all(log, '\\\\|poke\\\\|p2\\\\|([^\\\\|]+)', 1)"))
    
    # Then clean each array element to keep only the Pokémon name
    df = df.withColumn("player1_pokemon_array", 
                                 transform("player1_pokemon_array", lambda x: regexp_replace(x, ", L\\d+.*", "")))
    df = df.withColumn("player2_pokemon_array", 
                                 transform("player2_pokemon_array", lambda x: regexp_replace(x, ", L\\d+.*", "")))
    
    # Extract full team preview for both players
    df = df.withColumn("player1_full_team_raw", 
                               expr("regexp_extract_all(log, '\\\\|poke\\\\|p1\\\\|([^,]+)', 1)"))
    df = df.withColumn("player2_full_team_raw", 
                               expr("regexp_extract_all(log, '\\\\|poke\\\\|p2\\\\|([^,]+)', 1)"))
    
    # Clean the team arrays to handle any special forms (like Urshifu-*)
    df = df.withColumn("player1_full_team", 
                               transform("player1_full_team_raw", lambda x: regexp_replace(x, "\\-\\*", "")))
    df = df.withColumn("player2_full_team", 
                               transform("player2_full_team_raw", lambda x: regexp_replace(x, "\\-\\*", "")))
    
    
    
    # Extract all moves used by each player's Pokémon (will parse this later)
    df = df.withColumn("player1_moves_raw", 
                                 expr("regexp_extract_all(log, '\\\\|move\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    df = df.withColumn("player2_moves_raw", 
                                 expr("regexp_extract_all(log, '\\\\|move\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    
    
    # Extract items used by each Pokémon
    # logs_df = logs_df.withColumn("player1_item", regexp_extract("log", r"\|enditem\|p1b: [^\|]+", 1))
    # logs_df = logs_df.withColumn("player2_item", regexp_extract("log", r"\|enditem\|p2b: [^\|]+", 1))
    
    # # Extract Tera type used by each Pokémon
    # logs_df = logs_df.withColumn("player1_tera", regexp_extract("log", r"\|-terastallize\|p1a: [^\|]+\|([^\|]+)", 1))
    # logs_df = logs_df.withColumn("player2_tera", regexp_extract("log", r"\|-terastallize\|p2a: [^\|]+\|([^\|]+)", 1))
    
    # Extract items used by each Pokémon - improved version
    # df = df.withColumn("player1_items_raw", 
    #                            expr("regexp_extract_all(log, '\\\\|item\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    # df = df.withColumn("player2_items_raw", 
    #                            expr("regexp_extract_all(log, '\\\\|item\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    
    # # Also capture enditem events which show when items are consumed/lost
    # df = df.withColumn("player1_enditems_raw", 
    #                            expr("regexp_extract_all(log, '\\\\|enditem\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    # df = df.withColumn("player2_enditems_raw", 
    #                            expr("regexp_extract_all(log, '\\\\|enditem\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    
    # Extract Tera type used by each Pokémon - improved version
    # df = df.withColumn("player1_tera_raw", 
    #                            expr("regexp_extract_all(log, '\\\\|-terastallize\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    # df = df.withColumn("player2_tera_raw", 
    #                            expr("regexp_extract_all(log, '\\\\|-terastallize\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))

    # Drop the temporary columns before returning
    # final_df = df_with_team_info.drop("p1_team_details", "p2_team_details")
    
    return df


logs_df_with_teamsheet_raw = extract_raw_data(logs_df_with_teamsheet)

pandas_df_preview2 = logs_df_with_teamsheet_raw.limit(15).toPandas()

pandas_df_preview2


,id,format,formatid,log,password,players,private,rating,uploadtime,views,...,p1_pokemon_data_open_team,p2_pokemon_data_open_team,player1_pokemon_array,player2_pokemon_array,player1_full_team_raw,player2_full_team_raw,player1_full_team,player2_full_team,player1_moves_raw,player2_moves_raw
0,gen9vgc2025regg-2255482030,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆video gamah\n|j|☆goom_n_gloomba\n|t:|17331...,None,"[video gamah, goom_n_gloomba]",0,1064.0,1733179523,20,...,None,None,"[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...","[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...","[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu, Ogerpon-Hearthflame, Farig...","[|move|p1b: Kansas|Tailwind, |move|p1a: Red mi...","[|move|p2b: Farigiraf|Helping Hand, |move|p2a:..."
1,gen9vgc2025regg-2255265255,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆greenbean98\n|j|☆starkhd\n|t:|1733158007\n...,None,"[greenbean98, starkhd]",0,1049.0,1733158270,6,...,None,None,"[Zacian-*, Amoonguss, Incineroar, Flutter Mane...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[Zacian-*, Amoonguss, Incineroar, Flutter Mane...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[Zacian, Amoonguss, Incineroar, Flutter Mane, ...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[|move|p1a: Zacian|Protect, |move|p1b: Amoongu...","[|move|p2a: Grimmsnarl|Reflect, |move|p2b: Mir..."
2,gen9vgc2025regg-2254987654,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆tsilva22\n|j|☆aroswing\n|t:|1733117542\n|g...,None,"[tsilva22, aroswing]",0,1095.0,1733117753,3,...,None,None,"[Zacian-*, Iron Moth, Landorus-Therian, Whimsi...","[Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...","[Zacian-*, Iron Moth, Landorus-Therian, Whimsi...","[Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...","[Zacian, Iron Moth, Landorus-Therian, Whimsico...","[Urshifu, Kyogre, Tornadus, Toxicroak, Rhydon,...","[|move|p1a: Whimsicott|Tailwind, |move|p1b: La...","[|move|p2a: Tornadus|Bleakwind Storm, |move|p2..."
3,gen9vgc2025regg-2255274377,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Freaky Mane\n|j|☆Bhdidi\n|html|<table widt...,None,"[Freaky Mane, Bhdidi]",0,NaN,1733159149,3,...,"{'Flutter Mane': ('AssaultVest', 'Protosynthes...","{'Flutter Mane': ('ChoiceSpecs', 'Protosynthes...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[|move|p1a: Koraidon|Flare Blitz, |move|p1b: F...","[|move|p2b: Great Neck|Draco Meteor, |move|p2b..."
4,gen9vgc2025regg-2255415209,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆GriefersVGC\n|j|☆johnnydrama219\n|t:|17331...,None,"[GriefersVGC, johnnydrama219]",0,1049.0,1733172341,9,...,None,None,"[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[|move|p1a: Smeargle|Follow Me, |move|p1b: Flu...","[|move|p2b: Dragapult|Phantom Force, |move|p2a..."
5,gen9vgc2025regg-2254867190,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Mumu_the_Swampert\n|j|☆alejaji\n|t:|173310...,None,"[Mumu_the_Swampert, alejaji]",0,1085.0,1733100987,13,...,None,None,"[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[|move|p1a: Tornadus|Tailwind, |move|p1b: Pach...","[|move|p2b: Flutter Mane|Icy Wind, |move|p2a: ..."
6

Weather, Forfeits, Status Effects

In [6]:
# Extract weather, status, forfeit, winner
def extract_weather_status_forfeit_winner_changes(df):
    df = df.withColumn("weather", regexp_extract("log", r"\|weather\|([^\|]+)", 1))
    
    # Extract forfeit information
    df = df.withColumn("forfeit", regexp_extract("log", r"\|message\|([^\|]+ forfeited)", 1))
    
    # Extract status effects like confusion, sleep, etc.
    df = df.withColumn("status_effects", regexp_extract("log", r"\|start\|p1a: [^\|]+\|([^\|]+)", 1))

    df = df.withColumn("winner", regexp_extract("log", r"\|win\|([^\|]+)", 1))

    return df

logs_df_with_teamsheet_raw_weather = extract_weather_status_forfeit_winner_changes(logs_df_with_teamsheet_raw)

pandas_df_preview3 = logs_df_with_teamsheet_raw_weather.limit(15).toPandas()

pandas_df_preview3


,id,format,formatid,log,password,players,private,rating,uploadtime,views,...,player1_full_team_raw,player2_full_team_raw,player1_full_team,player2_full_team,player1_moves_raw,player2_moves_raw,weather,forfeit,status_effects,winner
0,gen9vgc2025regg-2255482030,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆video gamah\n|j|☆goom_n_gloomba\n|t:|17331...,None,"[video gamah, goom_n_gloomba]",0,1064.0,1733179523,20,...,"[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...","[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu, Ogerpon-Hearthflame, Farig...","[|move|p1b: Kansas|Tailwind, |move|p1a: Red mi...","[|move|p2b: Farigiraf|Helping Hand, |move|p2a:...",,,,video gamah\n
1,gen9vgc2025regg-2255265255,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆greenbean98\n|j|☆starkhd\n|t:|1733158007\n...,None,"[greenbean98, starkhd]",0,1049.0,1733158270,6,...,"[Zacian-*, Amoonguss, Incineroar, Flutter Mane...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[Zacian, Amoonguss, Incineroar, Flutter Mane, ...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[|move|p1a: Zacian|Protect, |move|p1b: Amoongu...","[|move|p2a: Grimmsnarl|Reflect, |move|p2b: Mir...",,,,starkhd\n
2,gen9vgc2025regg-2254987654,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆tsilva22\n|j|☆aroswing\n|t:|1733117542\n|g...,None,"[tsilva22, aroswing]",0,1095.0,1733117753,3,...,"[Zacian-*, Iron Moth, Landorus-Therian, Whimsi...","[Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...","[Zacian, Iron Moth, Landorus-Therian, Whimsico...","[Urshifu, Kyogre, Tornadus, Toxicroak, Rhydon,...","[|move|p1a: Whimsicott|Tailwind, |move|p1b: La...","[|move|p2a: Tornadus|Bleakwind Storm, |move|p2...",,,,aroswing\n
3,gen9vgc2025regg-2255274377,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Freaky Mane\n|j|☆Bhdidi\n|html|<table widt...,None,"[Freaky Mane, Bhdidi]",0,NaN,1733159149,3,...,"[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[|move|p1a: Koraidon|Flare Blitz, |move|p1b: F...","[|move|p2b: Great Neck|Draco Meteor, |move|p2b...",,,,Freaky Mane\n
4,gen9vgc2025regg-2255415209,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆GriefersVGC\n|j|☆johnnydrama219\n|t:|17331...,None,"[GriefersVGC, johnnydrama219]",0,1049.0,1733172341,9,...,"[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[|move|p1a: Smeargle|Follow Me, |move|p1b: Flu...","[|move|p2b: Dragapult|Phantom Force, |move|p2a...",,,,GriefersVGC\n
5,gen9vgc2025regg-2254867190,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Mumu_the_Swampert\n|j|☆alejaji\n|t:|173310...,None,"[Mumu_the_Swampert, alejaji]",0,1085.0,1733100987,13,...,"[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[|move|p1a: Tornadus|Tailwind, |move|p1b: Pach...","[|move|p2b: Flutter Mane|Icy Wind, |move|p2a: ...",,,,Mumu_the_Swampert\n
6,gen9vgc2025regg-2255426521,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆groovalishis\n|j|☆juacox300\n|t:|173317328...,None,"[groovalishis, juacox300]",0,1089.0,1733173539,51,...,"[Koraidon, Flutter Mane, Rillaboom, Gholdengo,...","[Iron Hands, Alcremie, Miraidon, Chi-Yu, Vikav...","[Koraidon, Flutter Mane, Rillaboom, Gholdengo,...","[Iron Hands, Alcremie, Miraidon, Chi-Yu, Vikav...","[|move|p1a: Flutter Mane|Moonblast, |move|p1b:...","[|move|p2b: Alcremie|Encore, |move|p2a: Iron H...",,,,juacox300\n
7,gen9vgc2025regg-2255448421,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Mr.Man1206\n|j|☆lukve_\n|t:|1733175480\n|g...,None,"[Mr.Man1206, lukve_]",0,1075.0,1733175763,10,...,"[Chi-Yu, Gholdengo, Koraidon, Flutter Mane, Ve...","[Inci

Match Outcome

In [7]:
# # Extract the winner
# logs_df = logs_df.withColumn("winner", regexp_extract("log", r"\|win\|([^\|]+)", 1))


Lets check our work!

In [8]:
logs_df_with_teamsheet_raw_weather.show(5)

+--------------------+--------------------+---------------+--------------------+--------+--------------------+-------+------+----------+-----+-----------+--------------+---------------------+---------------------+--------------------+--------------------+--------------------+-------------------------+-------------------------+---------------------+---------------------+---------------------+---------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+--------------+-------------+
|                  id|              format|       formatid|                 log|password|             players|private|rating|uploadtime|views|    player1|       player2|player1_rating_before|player2_rating_before|player1_rating_after|player2_rating_after|team_sheets_accepted|p1_pokemon_data_open_team|p2_pokemon_data_open_team|player1_pokemon_array|player2_pokemon_array|player1_full_team_raw|player2_full_team_raw|   player1_full_team|   player2_full_t

That's a bit hard to read, lets convert those top 5 rows to pandas and print it out.

In [9]:
import pandas as pd

# Convert to Pandas DataFrame (only do this for small result sets!)
pandas_df = logs_df_with_teamsheet_raw_weather.limit(5).toPandas()

# Set pandas display options to show full arrays and columns
# original_max_colwidth = pd.get_option('display.max_colwidth')

# Temporarily set max_colwidth to None for displaying
# pd.set_option('display.max_colwidth', None)

# Display only the pokemon array columns
print("Player 1 Pokemon Arrays:")
print(pandas_df[['player1_pokemon_array']])
print("\nPlayer 2 Pokemon Arrays:")
print(pandas_df[['player2_pokemon_array']])


# Display only the pokemon array columns
print("Player 1 Logs Team:")
print(pandas_df[['player1_full_team']])
print("\nPlayer 2 Pokemon Arrays:")
print(pandas_df[['player2_full_team']])


print("Player 1 Open Sheet Team:")
print(pandas_df[['p1_pokemon_data_open_team']])
print("\nPlayer 2 Open Sheet Team:")
print(pandas_df[['p2_pokemon_data_open_team']])

# Reset to original setting
# pd.set_option('display.max_colwidth', original_max_colwidth)


# Display the pandas DataFrame
pandas_df

Player 1 Pokemon Arrays:
                               player1_pokemon_array
0  [Eternatus, Tornadus, Ogerpon-Wellspring, Inci...
1  [Zacian-*, Amoonguss, Incineroar, Flutter Mane...
2  [Zacian-*, Iron Moth, Landorus-Therian, Whimsi...
3  [Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...
4  [Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...

Player 2 Pokemon Arrays:
                               player2_pokemon_array
0  [Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...
1  [Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...
2  [Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...
3  [Incineroar, Raging Bolt, Flutter Mane, Zamaze...
4  [Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...
Player 1 Logs Team:
                                   player1_full_team
0  [Eternatus, Tornadus, Ogerpon-Wellspring, Inci...
1  [Zacian, Amoonguss, Incineroar, Flutter Mane, ...
2  [Zacian, Iron Moth, Landorus-Therian, Whimsico...
3  [Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...
4  [Smeargle, Calyrex-Ice, U

,id,format,formatid,log,password,players,private,rating,uploadtime,views,...,player1_full_team_raw,player2_full_team_raw,player1_full_team,player2_full_team,player1_moves_raw,player2_moves_raw,weather,forfeit,status_effects,winner
0,gen9vgc2025regg-2255482030,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆video gamah\n|j|☆goom_n_gloomba\n|t:|17331...,None,"[video gamah, goom_n_gloomba]",0,1064.0,1733179523,20,...,"[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...","[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu, Ogerpon-Hearthflame, Farig...","[|move|p1b: Kansas|Tailwind, |move|p1a: Red mi...","[|move|p2b: Farigiraf|Helping Hand, |move|p2a:...",,,,video gamah\n
1,gen9vgc2025regg-2255265255,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆greenbean98\n|j|☆starkhd\n|t:|1733158007\n...,None,"[greenbean98, starkhd]",0,1049.0,1733158270,6,...,"[Zacian-*, Amoonguss, Incineroar, Flutter Mane...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[Zacian, Amoonguss, Incineroar, Flutter Mane, ...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[|move|p1a: Zacian|Protect, |move|p1b: Amoongu...","[|move|p2a: Grimmsnarl|Reflect, |move|p2b: Mir...",,,,starkhd\n
2,gen9vgc2025regg-2254987654,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆tsilva22\n|j|☆aroswing\n|t:|1733117542\n|g...,None,"[tsilva22, aroswing]",0,1095.0,1733117753,3,...,"[Zacian-*, Iron Moth, Landorus-Therian, Whimsi...","[Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...","[Zacian, Iron Moth, Landorus-Therian, Whimsico...","[Urshifu, Kyogre, Tornadus, Toxicroak, Rhydon,...","[|move|p1a: Whimsicott|Tailwind, |move|p1b: La...","[|move|p2a: Tornadus|Bleakwind Storm, |move|p2...",,,,aroswing\n
3,gen9vgc2025regg-2255274377,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Freaky Mane\n|j|☆Bhdidi\n|html|<table widt...,None,"[Freaky Mane, Bhdidi]",0,NaN,1733159149,3,...,"[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[|move|p1a: Koraidon|Flare Blitz, |move|p1b: F...","[|move|p2b: Great Neck|Draco Meteor, |move|p2b...",,,,Freaky Mane\n
4,gen9vgc2025regg-2255415209,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆GriefersVGC\n|j|☆johnnydrama219\n|t:|17331...,None,"[GriefersVGC, johnnydrama219]",0,1049.0,1733172341,9,...,"[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[|move|p1a: Smeargle|Follow Me, |move|p1b: Flu...","[|move|p2b: Dragapult|Phantom Force, |move|p2a...",,,,GriefersVGC\n


In [10]:
# # Extract all moves used by each player's Pokémon
# logs_df = logs_df.withColumn("player1_moves_raw", 
#                              expr("regexp_extract_all(log, '\\\\|move\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
# logs_df = logs_df.withColumn("player2_moves_raw", 
#                              expr("regexp_extract_all(log, '\\\\|move\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))

# Show the raw extracted move patterns to verify
# print("Raw move patterns for player 1:")
# pd.set_option('display.max_colwidth', None)
# print(logs_df.select("player1_moves_raw").limit(2).toPandas())

### Extract Pokemon Moves

Previously we built a column with all the data for a pokemon and its moves.

Now we will create a new df for the moves.

1. Explode the moves ie create a new row for each move
2. Extract the pokemon for each move, and its moves
3. Group each pokemon by the battle it took place in (because we seperated the dfs into a player 1 and player 2 we don't need to worry about 2 players using the same pokemon accidentally being joined together)
4. Convert to a map

In [11]:
from pyspark.sql.functions import regexp_extract, transform, expr, regexp_replace, col, explode, split, lit
from pyspark.sql.functions import collect_list, struct, map_from_entries, collect_set

def extract_pokemon_moves(df):
    # Explode the arrays to work with individual move patterns
    p1_moves_df = df.select("id", explode("player1_moves_raw").alias("move_pattern"))
    p2_moves_df = df.select("id", explode("player2_moves_raw").alias("move_pattern"))
    
    # Extract Pokémon and move from the patterns using regexp_extract
    p1_moves_df = p1_moves_df.withColumn("pokemon", 
                                         regexp_extract("move_pattern", "\\|move\\|p1[ab]: ([^\\|]+)\\|", 1))
    p1_moves_df = p1_moves_df.withColumn("move", 
                                         regexp_extract("move_pattern", "\\|move\\|p1[ab]: [^\\|]+\\|([^\\|]+)", 1))
    
    p2_moves_df = p2_moves_df.withColumn("pokemon", 
                                         regexp_extract("move_pattern", "\\|move\\|p2[ab]: ([^\\|]+)\\|", 1))
    p2_moves_df = p2_moves_df.withColumn("move", 
                                         regexp_extract("move_pattern", "\\|move\\|p2[ab]: [^\\|]+\\|([^\\|]+)", 1))
    
    # Group by battle ID and Pokémon to get all moves used by each Pokémon. 
    # We use set to remove duplicates (ie pokemon using same move more than once per match)
    p1_moves_by_pokemon = p1_moves_df.groupBy("id", "pokemon").agg(collect_set("move").alias("moves"))
    p2_moves_by_pokemon = p2_moves_df.groupBy("id", "pokemon").agg(collect_set("move").alias("moves"))
    
    # Convert to a map structure for easier joining
    p1_moves_map = p1_moves_by_pokemon.groupBy("id").agg(
        map_from_entries(collect_list(struct("pokemon", "moves"))).alias("player1_pokemon_moves")
    )
    p2_moves_map = p2_moves_by_pokemon.groupBy("id").agg(
        map_from_entries(collect_list(struct("pokemon", "moves"))).alias("player2_pokemon_moves")
    )
    
    # Join back to the main dataframe
    df = df.join(p1_moves_map, "id", "left")
    df = df.join(p2_moves_map, "id", "left")

    # drop moves raw since we don't need it anymore
    df = df.drop('player1_moves_raw', 'player2_moves_raw')

    return df


df_semi_final = extract_pokemon_moves(logs_df_with_teamsheet_raw)

pandas_df_preview4 = df_semi_final.limit(15).toPandas()

pandas_df_preview4


,id,format,formatid,log,password,players,private,rating,uploadtime,views,...,p1_pokemon_data_open_team,p2_pokemon_data_open_team,player1_pokemon_array,player2_pokemon_array,player1_full_team_raw,player2_full_team_raw,player1_full_team,player2_full_team,player1_pokemon_moves,player2_pokemon_moves
0,gen9vgc2025regg-2255482030,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆video gamah\n|j|☆goom_n_gloomba\n|t:|17331...,None,"[video gamah, goom_n_gloomba]",0,1064.0,1733179523,20,...,None,None,"[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...","[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...","[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu, Ogerpon-Hearthflame, Farig...","{'Kansas': ['Tailwind'], 'Red mist': ['Sludge ...","{'Farigiraf': ['Helping Hand', 'Psychic Noise'..."
1,gen9vgc2025regg-2255265255,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆greenbean98\n|j|☆starkhd\n|t:|1733158007\n...,None,"[greenbean98, starkhd]",0,1049.0,1733158270,6,...,None,None,"[Zacian-*, Amoonguss, Incineroar, Flutter Mane...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[Zacian-*, Amoonguss, Incineroar, Flutter Mane...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[Zacian, Amoonguss, Incineroar, Flutter Mane, ...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","{'Flutter Mane': ['Dazzling Gleam', 'Moonblast...","{'Grimmsnarl': ['Reflect', 'Spirit Break', 'Th..."
2,gen9vgc2025regg-2254987654,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆tsilva22\n|j|☆aroswing\n|t:|1733117542\n|g...,None,"[tsilva22, aroswing]",0,1095.0,1733117753,3,...,None,None,"[Zacian-*, Iron Moth, Landorus-Therian, Whimsi...","[Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...","[Zacian-*, Iron Moth, Landorus-Therian, Whimsi...","[Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...","[Zacian, Iron Moth, Landorus-Therian, Whimsico...","[Urshifu, Kyogre, Tornadus, Toxicroak, Rhydon,...","{'Kingambit': ['Sucker Punch'], 'Zacian': ['Pl...","{'Kyogre': ['Origin Pulse'], 'Tornadus': ['Ble..."
3,gen9vgc2025regg-2255274377,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Freaky Mane\n|j|☆Bhdidi\n|html|<table widt...,None,"[Freaky Mane, Bhdidi]",0,NaN,1733159149,3,...,"{'Flutter Mane': ('AssaultVest', 'Protosynthes...","{'Flutter Mane': ('ChoiceSpecs', 'Protosynthes...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","{'Koraidon': ['Flare Blitz'], 'Biraidon': ['Th...","{'Mismagius?!?': ['Dazzling Gleam'], 'Great Ne..."
4,gen9vgc2025regg-2255415209,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆GriefersVGC\n|j|☆johnnydrama219\n|t:|17331...,None,"[GriefersVGC, johnnydrama219]",0,1049.0,1733172341,9,...,None,None,"[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","{'Calyrex': ['Glacial Lance'], 'Flutter Mane':...","{'Sneasler': ['Close Combat'], 'Mewtwo': ['Exp..."
5,gen9vgc2025regg-2254867190,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Mumu_the_Swampert\n|j|☆alejaji\n|t:|173310...,None,"[Mumu_the_Swampert, alejaji]",0,1085.0,1733100987,13,...,None,None,"[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","{'Calyrex': ['Astral Barrage'], 'Tornadus': ['...","{'Flutter Mane': ['Dazzling Gleam', 'Icy Win

#### Cache the logs df so we don't have to do these expensive computes

In [12]:
# Cache the DataFrame after the expensive move mapping operations
logs_df = logs_df.cache()

### Process Items

Each pokemon holds an item. This item is not revealed unless some game event leads to it being revealed.

To solve this, we parse all item logs to match items to a pokemon. This could be an item knocked off mid turn, or an end turn item useage like berry consumption.

In [13]:

from pyspark.sql.functions import regexp_extract, transform, expr, regexp_replace, col, explode, split, lit
from pyspark.sql.functions import collect_list, struct, map_from_entries, collect_set, explode_outer

# Process both enditem events and item events

def extract_items(df):
    # First, extract all item-related patterns from the logs
    df = df.withColumn("player1_item_patterns", 
                               expr("regexp_extract_all(log, '\\\\|-item\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    df = df.withColumn("player2_item_patterns", 
                               expr("regexp_extract_all(log, '\\\\|-item\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    
    # Also capture enditem events which show when items are consumed/lost
    df = df.withColumn("player1_enditem_patterns", 
                               expr("regexp_extract_all(log, '\\\\|-enditem\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    df = df.withColumn("player2_enditem_patterns", 
                               expr("regexp_extract_all(log, '\\\\|-enditem\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    
    # Process item events for player 1
    p1_item_df = df.select("id", explode_outer("player1_item_patterns").alias("item_pattern"))
    p1_item_df = p1_item_df.withColumn("pokemon", 
                                     regexp_extract("item_pattern", "\\|-item\\|p1[ab]: ([^\\|]+)\\|", 1))
    p1_item_df = p1_item_df.withColumn("item", 
                                     regexp_extract("item_pattern", "\\|-item\\|p1[ab]: [^\\|]+\\|([^\\|]+)", 1))
    
    # Process enditem events for player 1
    p1_enditem_df = df.select("id", explode_outer("player1_enditem_patterns").alias("enditem_pattern"))
    p1_enditem_df = p1_enditem_df.withColumn("pokemon", 
                                           regexp_extract("enditem_pattern", "\\|-enditem\\|p1[ab]: ([^\\|]+)\\|", 1))
    p1_enditem_df = p1_enditem_df.withColumn("item", 
                                           regexp_extract("enditem_pattern", "\\|-enditem\\|p1[ab]: [^\\|]+\\|([^\\|]+)", 1))
    
    # Process item events for player 2
    p2_item_df = df.select("id", explode_outer("player2_item_patterns").alias("item_pattern"))
    p2_item_df = p2_item_df.withColumn("pokemon", 
                                     regexp_extract("item_pattern", "\\|-item\\|p2[ab]: ([^\\|]+)\\|", 1))
    p2_item_df = p2_item_df.withColumn("item", 
                                     regexp_extract("item_pattern", "\\|-item\\|p2[ab]: [^\\|]+\\|([^\\|]+)", 1))
    
    # Process enditem events for player 2
    p2_enditem_df = df.select("id", explode_outer("player2_enditem_patterns").alias("enditem_pattern"))
    p2_enditem_df = p2_enditem_df.withColumn("pokemon", 
                                           regexp_extract("enditem_pattern", "\\|-enditem\\|p2[ab]: ([^\\|]+)\\|", 1))
    p2_enditem_df = p2_enditem_df.withColumn("item", 
                                           regexp_extract("enditem_pattern", "\\|-enditem\\|p2[ab]: [^\\|]+\\|([^\\|]+)", 1))
    
    # Union the item and enditem dataframes for each player
    p1_all_items_df = p1_item_df.union(p1_enditem_df)
    p2_all_items_df = p2_item_df.union(p2_enditem_df)
    
    # Group by battle ID and Pokémon to get items used by each Pokémon
    p1_items_by_pokemon = p1_all_items_df.filter(col("pokemon").isNotNull() & col("item").isNotNull()) \
                                        .groupBy("id", "pokemon").agg(collect_set("item").alias("items"))
    p2_items_by_pokemon = p2_all_items_df.filter(col("pokemon").isNotNull() & col("item").isNotNull()) \
                                        .groupBy("id", "pokemon").agg(collect_set("item").alias("items"))
    
    # Convert to a map structure for easier joining
    p1_items_map = p1_items_by_pokemon.groupBy("id").agg(
        map_from_entries(collect_list(struct("pokemon", "items"))).alias("player1_pokemon_items")
    )
    p2_items_map = p2_items_by_pokemon.groupBy("id").agg(
        map_from_entries(collect_list(struct("pokemon", "items"))).alias("player2_pokemon_items")
    )
    
    # Join back to the main dataframe
    df = df.join(p1_items_map, "id", "left")
    df = df.join(p2_items_map, "id", "left")

    df = df.drop('player1_item_patterns', 'player2_item_patterns', 'player1_enditem_patterns', 'player2_enditem_patterns')

    return df


df_semi_semi_final = extract_items(df_semi_final)

pandas_df_preview5 = df_semi_semi_final.limit(15).toPandas()

pandas_df_preview5
# # Display the results to verify
# sample_df = logs_df.select("id", "player1_pokemon_items", "player2_pokemon_items").limit(2)
# print(sample_df.toPandas())
# # // ... existing code ...

,id,format,formatid,log,password,players,private,rating,uploadtime,views,...,player1_pokemon_array,player2_pokemon_array,player1_full_team_raw,player2_full_team_raw,player1_full_team,player2_full_team,player1_pokemon_moves,player2_pokemon_moves,player1_pokemon_items,player2_pokemon_items
0,gen9vgc2025regg-2255482030,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆video gamah\n|j|☆goom_n_gloomba\n|t:|17331...,None,"[video gamah, goom_n_gloomba]",0,1064.0,1733179523,20,...,"[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...","[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...","[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu, Ogerpon-Hearthflame, Farig...","{'Kansas': ['Tailwind'], 'Red mist': ['Sludge ...","{'Farigiraf': ['Helping Hand', 'Psychic Noise'...",{'Nuclear Bomb': ['Booster Energy ']},{'Farigiraf': ['Electric Seed ']}
1,gen9vgc2025regg-2255265255,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆greenbean98\n|j|☆starkhd\n|t:|1733158007\n...,None,"[greenbean98, starkhd]",0,1049.0,1733158270,6,...,"[Zacian-*, Amoonguss, Incineroar, Flutter Mane...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[Zacian-*, Amoonguss, Incineroar, Flutter Mane...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[Zacian, Amoonguss, Incineroar, Flutter Mane, ...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","{'Flutter Mane': ['Dazzling Gleam', 'Moonblast...","{'Grimmsnarl': ['Reflect', 'Spirit Break', 'Th...",{'Flutter Mane': ['Booster Energy ']},{'Farigiraf': ['Electric Seed ']}
2,gen9vgc2025regg-2254987654,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆tsilva22\n|j|☆aroswing\n|t:|1733117542\n|g...,None,"[tsilva22, aroswing]",0,1095.0,1733117753,3,...,"[Zacian-*, Iron Moth, Landorus-Therian, Whimsi...","[Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...","[Zacian-*, Iron Moth, Landorus-Therian, Whimsi...","[Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...","[Zacian, Iron Moth, Landorus-Therian, Whimsico...","[Urshifu, Kyogre, Tornadus, Toxicroak, Rhydon,...","{'Kingambit': ['Sucker Punch'], 'Zacian': ['Pl...","{'Kyogre': ['Origin Pulse'], 'Tornadus': ['Ble...",None,None
3,gen9vgc2025regg-2255274377,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Freaky Mane\n|j|☆Bhdidi\n|html|<table widt...,None,"[Freaky Mane, Bhdidi]",0,NaN,1733159149,3,...,"[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","{'Koraidon': ['Flare Blitz'], 'Biraidon': ['Th...","{'Mismagius?!?': ['Dazzling Gleam'], 'Great Ne...",{'wolfeglickreal!!': ['Sitrus Berry']},None
4,gen9vgc2025regg-2255415209,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆GriefersVGC\n|j|☆johnnydrama219\n|t:|17331...,None,"[GriefersVGC, johnnydrama219]",0,1049.0,1733172341,9,...,"[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","{'Calyrex': ['Glacial Lance'], 'Flutter Mane':...","{'Sneasler': ['Close Combat'], 'Mewtwo': ['Exp...",{'Smeargle': ['Focus Sash ']},"{'Sneasler': ['Psychic Seed '], 'Indeedee': ['..."
5,gen9vgc2025regg-2254867190,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Mumu_the_Swampert\n|j|☆alejaji\n|t:|173310...,None,"[Mumu_the_Swampert, alejaji]",0,1085.0,1733100987,13,...,"[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wel

Lets do a pandas print for readbility

In [14]:
# Convert to Pandas DataFrame (only do this for small result sets!)
pandas_df = logs_df.limit(5).toPandas()


# Display the pandas DataFrame
pandas_df

,format,formatid,id,log,password,players,private,rating,uploadtime,views,player1,player2,player1_rating_before,player2_rating_before,player1_rating_after,player2_rating_after
0,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2255482030,|j|☆video gamah\n|j|☆goom_n_gloomba\n|t:|17331...,None,"[video gamah, goom_n_gloomba]",0,1064.0,1733179523,20,video gamah,goom_n_gloomba,,,,
1,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2255265255,|j|☆greenbean98\n|j|☆starkhd\n|t:|1733158007\n...,None,"[greenbean98, starkhd]",0,1049.0,1733158270,6,greenbean98,starkhd,,,,
2,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2254987654,|j|☆tsilva22\n|j|☆aroswing\n|t:|1733117542\n|g...,None,"[tsilva22, aroswing]",0,1095.0,1733117753,3,tsilva22,aroswing,,,,
3,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2255274377,|j|☆Freaky Mane\n|j|☆Bhdidi\n|html|<table widt...,None,"[Freaky Mane, Bhdidi]",0,NaN,1733159149,3,Freaky Mane,Bhdidi,,,,
4,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2255415209,|j|☆GriefersVGC\n|j|☆johnnydrama219\n|t:|17331...,None,"[GriefersVGC, johnnydrama219]",0,1049.0,1733172341,9,GriefersVGC,johnnydrama219,,,,


### Extracting Tera Information

In [15]:
from pyspark.sql.functions import regexp_extract, transform, expr, regexp_replace, col, explode, split, lit
from pyspark.sql.functions import collect_list, struct, map_from_entries, collect_set, explode_outer

# Process terastallize events for both players
# First, extract all terastallize patterns from the logs
def extract_tera(df):
    df = df.withColumn("player1_tera_patterns", 
                               expr("regexp_extract_all(log, '\\\\|-terastallize\\\\|p1[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    df = df.withColumn("player2_tera_patterns", 
                               expr("regexp_extract_all(log, '\\\\|-terastallize\\\\|p2[ab]: ([^\\\\|]+)\\\\|([^\\\\|]+)', 0)"))
    
    # Process terastallize events for player 1
    p1_tera_df = df.select("id", explode_outer("player1_tera_patterns").alias("tera_pattern"))
    p1_tera_df = p1_tera_df.withColumn("pokemon", 
                                     regexp_extract("tera_pattern", "\\|-terastallize\\|p1[ab]: ([^\\|]+)\\|", 1))
    p1_tera_df = p1_tera_df.withColumn("tera_type", 
                                     regexp_extract("tera_pattern", "\\|-terastallize\\|p1[ab]: [^\\|]+\\|([^\\|\\n]+)", 1))
    
    # Process terastallize events for player 2
    p2_tera_df = df.select("id", explode_outer("player2_tera_patterns").alias("tera_pattern"))
    p2_tera_df = p2_tera_df.withColumn("pokemon", 
                                     regexp_extract("tera_pattern", "\\|-terastallize\\|p2[ab]: ([^\\|]+)\\|", 1))
    p2_tera_df = p2_tera_df.withColumn("tera_type", 
                                     regexp_extract("tera_pattern", "\\|-terastallize\\|p2[ab]: [^\\|]+\\|([^\\|\\n]+)", 1))
    
    # Filter out null values and create maps
    p1_tera_map = p1_tera_df.filter(col("pokemon").isNotNull() & col("tera_type").isNotNull()) \
                           .groupBy("id") \
                           .agg(map_from_entries(collect_list(struct("pokemon", "tera_type"))).alias("player1_pokemon_tera"))
    
    p2_tera_map = p2_tera_df.filter(col("pokemon").isNotNull() & col("tera_type").isNotNull()) \
                           .groupBy("id") \
                           .agg(map_from_entries(collect_list(struct("pokemon", "tera_type"))).alias("player2_pokemon_tera"))
    
    # Join back to the main dataframe
    df = df.join(p1_tera_map, "id", "left")
    df = df.join(p2_tera_map, "id", "left")

    # drop raw patterns
    df = df.drop('player1_tera_patterns', 'player2_tera_patterns')

    return df


df_final = extract_tera(df_semi_semi_final)

pandas_df_preview6 = df_final.limit(15).toPandas()

pandas_df_preview6

,id,format,formatid,log,password,players,private,rating,uploadtime,views,...,player1_full_team_raw,player2_full_team_raw,player1_full_team,player2_full_team,player1_pokemon_moves,player2_pokemon_moves,player1_pokemon_items,player2_pokemon_items,player1_pokemon_tera,player2_pokemon_tera
0,gen9vgc2025regg-2255482030,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆video gamah\n|j|☆goom_n_gloomba\n|t:|17331...,None,"[video gamah, goom_n_gloomba]",0,1064.0,1733179523,20,...,"[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu-*, Ogerpon-Hearthflame, Far...","[Eternatus, Tornadus, Ogerpon-Wellspring, Inci...","[Miraidon, Urshifu, Ogerpon-Hearthflame, Farig...","{'Kansas': ['Tailwind'], 'Red mist': ['Sludge ...","{'Farigiraf': ['Helping Hand', 'Psychic Noise'...",{'Nuclear Bomb': ['Booster Energy ']},{'Farigiraf': ['Electric Seed ']},{'Nuclear Bomb': 'Fairy'},{'Ogerpon': 'Fire'}
1,gen9vgc2025regg-2255265255,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆greenbean98\n|j|☆starkhd\n|t:|1733158007\n...,None,"[greenbean98, starkhd]",0,1049.0,1733158270,6,...,"[Zacian-*, Amoonguss, Incineroar, Flutter Mane...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","[Zacian, Amoonguss, Incineroar, Flutter Mane, ...","[Miraidon, Iron Hands, Iron Bundle, Chi-Yu, Gr...","{'Flutter Mane': ['Dazzling Gleam', 'Moonblast...","{'Grimmsnarl': ['Reflect', 'Spirit Break', 'Th...",{'Flutter Mane': ['Booster Energy ']},{'Farigiraf': ['Electric Seed ']},None,{'Miraidon': 'Electric'}
2,gen9vgc2025regg-2254987654,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆tsilva22\n|j|☆aroswing\n|t:|1733117542\n|g...,None,"[tsilva22, aroswing]",0,1095.0,1733117753,3,...,"[Zacian-*, Iron Moth, Landorus-Therian, Whimsi...","[Urshifu-*, Kyogre, Tornadus, Toxicroak, Rhydo...","[Zacian, Iron Moth, Landorus-Therian, Whimsico...","[Urshifu, Kyogre, Tornadus, Toxicroak, Rhydon,...","{'Kingambit': ['Sucker Punch'], 'Zacian': ['Pl...","{'Kyogre': ['Origin Pulse'], 'Tornadus': ['Ble...",None,None,None,{'Tornadus': 'Dark'}
3,gen9vgc2025regg-2255274377,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Freaky Mane\n|j|☆Bhdidi\n|html|<table widt...,None,"[Freaky Mane, Bhdidi]",0,NaN,1733159149,3,...,"[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","[Flutter Mane, Koraidon, Iron Valiant, Chi-Yu,...","[Incineroar, Raging Bolt, Flutter Mane, Zamaze...","{'Koraidon': ['Flare Blitz'], 'Biraidon': ['Th...","{'Mismagius?!?': ['Dazzling Gleam'], 'Great Ne...",{'wolfeglickreal!!': ['Sitrus Berry']},None,{'Freaky Mane': 'Fairy'},{'Great Neck': 'Fairy'}
4,gen9vgc2025regg-2255415209,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆GriefersVGC\n|j|☆johnnydrama219\n|t:|17331...,None,"[GriefersVGC, johnnydrama219]",0,1049.0,1733172341,9,...,"[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","[Smeargle, Calyrex-Ice, Ursaluna-Bloodmoon, To...","[Mewtwo, Indeedee-F, Iron Hands, Dragapult, Sn...","{'Calyrex': ['Glacial Lance'], 'Flutter Mane':...","{'Sneasler': ['Close Combat'], 'Mewtwo': ['Exp...",{'Smeargle': ['Focus Sash ']},"{'Sneasler': ['Psychic Seed '], 'Indeedee': ['...",{'Flutter Mane': 'Fairy'},{'Mewtwo': 'Psychic'}
5,gen9vgc2025regg-2254867190,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆Mumu_the_Swampert\n|j|☆alejaji\n|t:|173310...,None,"[Mumu_the_Swampert, alejaji]",0,1085.0,1733100987,13,...,"[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","[Calyrex-Shadow, Pachirisu, Rillaboom, Urshifu...","[Miraidon, Chi-Yu, Ogerpon-Wellspring, Flutter...","{'Calyrex': ['Astral Barrage'], 'Tornadus': ['...","{'Flutter Mane': ['Dazzling Gleam', 'Icy Wind'...",None,{'Flutter Mane': ['Booster Energy ']},{'Tornadus': 'Grass'},{'Urshifu': 'Dark'}
6,gen9vgc2025regg-2255426521,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,|j|☆groovalishis\n|j|☆juacox300\n|t:|173317328...,None,"[groovalishis, juacox300]",0,1089.0,1733173539,51,...,"[Koraidon, Flutter Mane, Rillaboom, Gh

In [16]:
# Convert to Pandas DataFrame (only do this for small result sets!)
pandas_df = logs_df.limit(5).toPandas()


# Display the pandas DataFrame
pandas_df

,format,formatid,id,log,password,players,private,rating,uploadtime,views,player1,player2,player1_rating_before,player2_rating_before,player1_rating_after,player2_rating_after
0,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2255482030,|j|☆video gamah\n|j|☆goom_n_gloomba\n|t:|17331...,None,"[video gamah, goom_n_gloomba]",0,1064.0,1733179523,20,video gamah,goom_n_gloomba,,,,
1,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2255265255,|j|☆greenbean98\n|j|☆starkhd\n|t:|1733158007\n...,None,"[greenbean98, starkhd]",0,1049.0,1733158270,6,greenbean98,starkhd,,,,
2,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2254987654,|j|☆tsilva22\n|j|☆aroswing\n|t:|1733117542\n|g...,None,"[tsilva22, aroswing]",0,1095.0,1733117753,3,tsilva22,aroswing,,,,
3,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2255274377,|j|☆Freaky Mane\n|j|☆Bhdidi\n|html|<table widt...,None,"[Freaky Mane, Bhdidi]",0,NaN,1733159149,3,Freaky Mane,Bhdidi,,,,
4,[Gen 9] VGC 2025 Reg G,gen9vgc2025regg,gen9vgc2025regg-2255415209,|j|☆GriefersVGC\n|j|☆johnnydrama219\n|t:|17331...,None,"[GriefersVGC, johnnydrama219]",0,1049.0,1733172341,9,GriefersVGC,johnnydrama219,,,,


### Save to parquet

We will end this exploration and save our results to parquet so we can reuse (and also put other functions so we can copy and paste them for later)

In [17]:
# ... existing code ...

# Save the processed data to parquet
from datetime import datetime
import os

# Create a timestamp for the processed data
process_date = datetime.now().strftime("%Y-%m-%d")

# Define the output paths
processed_data_path = f"../../data/processed/replays/{process_date}"
processed_ids_path = f"../../data/processed/replay_ids/{process_date}"

# Ensure directories exist
os.makedirs(os.path.dirname(processed_data_path), exist_ok=True)
os.makedirs(os.path.dirname(processed_ids_path), exist_ok=True)

# Save the full processed dataframe as parquet
logs_df.write.mode("overwrite").parquet(processed_data_path)
print(f"Saved processed data to {processed_data_path}")

# Save just the IDs of processed replays for future reference
id_df = logs_df.select("id")
id_df.write.mode("overwrite").parquet(processed_ids_path)
print(f"Saved processed IDs to {processed_ids_path}")

# Example of how to use these IDs for incremental processing in the future:
print("\nExample code for future incremental processing:")
print("""
# Load previously processed IDs
processed_ids = spark.read.parquet("../../data/processed/replay_ids/*")

# Load new raw data
new_logs_df = spark.read.option("recursiveFileLookup", "true").option("pathGlobFilter", "*.json").json("../../data/replays/gen9vgc2025regg/")

# Filter out already processed replays
new_logs_df = new_logs_df.join(processed_ids, "id", "left_anti")

# Process only the new data
# ... processing code ...

# Append to existing processed data
new_logs_df.write.mode("append").parquet("../../data/processed/replays/YYYY-MM-DD")

# Update the processed IDs
new_logs_df.select("id").write.mode("append").parquet("../../data/processed/replay_ids/YYYY-MM-DD")
""")

# For testing, let's see how many replays we processed
replay_count = logs_df.count()
print(f"\nProcessed {replay_count} replays in this run")

Saved processed data to ../../data/processed/replays/2025-03-23
Saved processed IDs to ../../data/processed/replay_ids/2025-03-23

Example code for future incremental processing:

# Load previously processed IDs
processed_ids = spark.read.parquet("../../data/processed/replay_ids/*")

# Load new raw data
new_logs_df = spark.read.option("recursiveFileLookup", "true").option("pathGlobFilter", "*.json").json("../../data/replays/gen9vgc2025regg/")

# Filter out already processed replays
new_logs_df = new_logs_df.join(processed_ids, "id", "left_anti")

# Process only the new data
# ... processing code ...

# Append to existing processed data
new_logs_df.write.mode("append").parquet("../../data/processed/replays/YYYY-MM-DD")

# Update the processed IDs
new_logs_df.select("id").write.mode("append").parquet("../../data/processed/replay_ids/YYYY-MM-DD")


Processed 689 replays in this run
